In [2]:
import pandas as pd
import numpy as np
from metrics import Pearson
from collections import OrderedDict
from keras.utils import to_categorical
from keras.models import load_model
from utils import prepare_data



In [3]:
def one_hot_emb(data: pd.DataFrame) -> pd.DataFrame:
    mapping = {
        'A': 0,
        'C': 1,
        'G': 2,
        'T': 3
    }
    one_hot_encode_lam = lambda seq: to_categorical([mapping[x] for x in seq])
    return data.apply(one_hot_encode_lam)

In [21]:
"""
    Integrated Gradients
"""
from motif_search import motif_utilities

model_name = "Baseline_m6A"

model = load_model(f'trained_models/{model_name}.h5',compile=False)

input_shape = model.get_config()["layers"][0]["config"]["batch_input_shape"]

y_size = input_shape[2]

5


In [3]:
train_data, valid_data, test_data = prepare_data()

max_seq_len = train_data['seq'].apply(lambda x: len(x)).max()

max_seq_len = 20788

train_data['length'] = train_data.seq.str.len()

train_data = train_data[train_data.length <= max_seq_len]

data_one_hot = one_hot_emb(train_data['seq'])
#data_struct = train_data['struct']

#padded_sequences = np.zeros((len(train_data['seq']), max_seq_len, 6), dtype=np.float32)
padded_sequences = np.zeros((len(train_data['seq']), max_seq_len, 4), dtype=np.float32)

indices = np.arange(train_data.shape[0])

for i, idx in enumerate(indices):
    #tmp = data_struct.iloc[idx]
    #padded_mask_struct = np.expand_dims(np.array(tmp != 'nan'), axis=1)
    #tmp[tmp == 'nan'] = -1
    #padded_struct = np.expand_dims(tmp, axis=1).astype('float64')
    seq_data = data_one_hot.iloc[idx]
    #seq_data = np.concatenate([seq_data, padded_mask_struct, padded_struct], axis=1)
    padded_sequences[i, -len(data_one_hot.iloc[idx]):, :] = seq_data

data_one_hot = padded_sequences

In [ ]:
print(gradients[6])
# print(data_one_hot)

seq_gradient = gradients[6][::-1]

for i in range(10):
   motif_utilities.plot_weights(seq_gradient[i*50:(i+1)*50], subticks_frequency=10)